In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [3]:
# 방법 1  from_template() 메소드에서 PromptTemplate 객체 생성

from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [4]:
prompt = prompt.format(country = "대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [6]:
chain.invoke({"country" : "대한민국"}).content

'대한민국의 수도는 서울입니다.'

In [7]:
# 방법 2 - PromptTemplate 객체 생성과 동시에 prompt 생성

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_varibles=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [8]:
prompt.format(country='대한민국')

'대한민국의 수도는 어디인가요?'

In [9]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template = template,
    input_variables=["country1"],
    partial_variables={
        "country2" : "미국"
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [10]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [11]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [13]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [14]:
chain = prompt_partial | llm

chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [15]:
chain.invoke({"country1":"대한민국", "country2":"호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

#### partial_variables : 부분 변수 채움
    - partial 사용하는 일반적인 용도 : 함수 부분적으로 사용 (항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우)
    대표적인 예 : 날짜/시간

In [16]:
from datetime import datetime

datetime.now().strftime("%B %d")

'September 27'

In [17]:
def get_today():
    return datetime.now().strftime("%B %d")

In [18]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요.",
    input_variable = ["n"],
    partial_variables={
        "today" : get_today
    },
)

prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x11964ee80>}, template='오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요.')

In [19]:
prompt.format(n=3)

'오늘의 날짜는 September 27 입니다. 오늘이 생일인 유명인 3명을 나열해주세요. 생년월일을 표기해주세요.'

In [20]:
chain = prompt | llm

In [21]:
print(chain.invoke(3).content)

1. Gwyneth Paltrow (1972년 9월 27일)
2. Lil Wayne (1982년 9월 27일)
3. Avril Lavigne (1984년 9월 27일)


In [22]:
print(chain.invoke({"today" : "Jan 02", "n" : 3}).content)

1. Kate Bosworth - 1983년 1월 2일
2. Taye Diggs - 1971년 1월 2일
3. Tia Carrere - 1967년 1월 2일


In [23]:
chain = prompt | ChatOpenAI(model_name="gpt-4o", temperature=0)

print(chain.invoke({"today" : "Jan 02", "n" : 3}).content)

#### 파일로부터 prompt 읽기(yaml 파일)

In [ ]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

In [ ]:
prompt.format(fruit="사과")

In [ ]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) | StrOutputParser()

In [ ]:
for token in chain.stream({"country" : "대한민국"}):
    print(token, end="", flush=True)

### ChatPromptTemplate

    - 대화목록을 프롬프트로 주입하고자 할 때 활용함
    - 메시지는 튜플(tuple) 혀식으로 구성(role, message)로 구성하여 리스트로 생성

    [role]
    - "system" : 시스템 설정 메시지, 주로 전역설정과 관련된 프롬프트
    - "human" : 사용자 입력 메시지
    - "ai" : ai 답변 메시지

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

In [ ]:
chat_prompt.messages

In [ ]:
chat_prompt.format(country="대한민국")

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다"),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"

)

messages

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [ ]:
llm.invoke(messages).content

In [ ]:
chain = chat_template | llm

In [ ]:
chain.invoke({"name" : "AI핑", "user_input" : "당신의 이름은 무엇입니까?"}).content

## MessagePlaceholder

    - Langchain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder를 제공함
    메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다"),
        ]
)

chat_prompt

In [ ]:
# conversation 대화목록을 나중에 추가하고자 할 때 MessagePlaceholder를 사용할 수 있음

chat_prompt.format(
    word_count=5,
    conversation = [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 AI핑입니다. 만나서 반갑습니다"),
        ("ai","반가워요! 앞으로 잘 부탁 드립니다")
    ]
)

In [ ]:
chat_prompt

In [ ]:
chain = chat_prompt | llm

chain

In [ ]:
result = chain.invoke(
    {
        "word_count":5,
        "conversation": [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 AI핑입니다. 만나서 반갑습니다"),
        ("ai","반가워요! 앞으로 잘 부탁 드립니다")
    ],
    }
)

result

In [ ]:
result.content

### FewShotPrompt Template 

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model_name = 'gpt-3.5-turbo'
)

question = "대한민국의 수도는?"

print(llm.invoke(question).content)

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [ ]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]